# Solutions to Agave OAuth Exercises


In [4]:
# import the requests library
import requests

# import getpass to prompt for a password
from getpass import getpass

In [5]:
# the base URL for interacting with the Agave API
base_url = 'https://api.tacc.utexas.edu'

In [6]:
# Set up your TACC credentials. Modify the username appropriately
username = 'jstubbs'
password = getpass(prompt='Hello {}. Please enter your TACC password: '.format(username))

Hello jstubbs. Please enter your TACC password: ········


In [16]:
rsp = requests.get(url='{}/clients/v2'.format(base_url), auth=(username, password))
rsp.status_code

200

In [17]:
# the clients service, like all Agave services, returns us JSON:
rsp.json()

{'message': 'Clients retrieved successfully.',
 'result': [{'_links': {'self': {'href': 'https://api.tacc.utexas.edu/clients/v2/DefaultApplication'},
    'subscriber': {'href': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs'},
    'subscriptions': {'href': 'https://api.tacc.utexas.edu/clients/v2/DefaultApplication/subscriptions/'}},
   'callbackUrl': None,
   'consumerKey': 'VzN6Cxj9GirfMsINqUVQxbhrQSQa',
   'description': None,
   'name': 'DefaultApplication',
   'tier': 'Unlimited'},
  {'_links': {'self': {'href': 'https://api.tacc.utexas.edu/clients/v2/postman'},
    'subscriber': {'href': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs'},
    'subscriptions': {'href': 'https://api.tacc.utexas.edu/clients/v2/postman/subscriptions/'}},
   'callbackUrl': 'https://www.getpostman.com/oauth2/callback',
   'consumerKey': 'fYEbfnaxo4LbqShSebng4f5LD18a',
   'description': '',
   'name': 'postman',
   'tier': 'Unlimited'},
  {'_links': {'self': {'href': 'https://api.tacc.utexas.edu/client

To create a simple OAuth client that has access to all basic Agave APIs, we need to make a POST request to the clients service. The only required field we need to pass in is `clientName` to give a name to our client. Each client we create must have a unique name.

In [18]:
# Pick a name for your client; this name will have to be different every time you run this cell. Otherwise, you
# will try to recreate a client with the same name and you will get an error.
client_name = 'cic_institute'

# make a POST request to the client's service, passing only that field. 
# Note that the parameter name uses camel case
data = {'clientName': client_name}
rsp = requests.post(url='{}/clients/v2'.format(base_url), data=data, auth=(username, password))
rsp.status_code

201

In [19]:
rsp.json()

{'message': 'Client created successfully.',
 'result': {'_links': {'self': {'href': 'https://api.tacc.utexas.edu/clients/v2/cic_institute'},
   'subscriber': {'href': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs'},
   'subscriptions': {'href': 'https://api.tacc.utexas.edu/clients/v2/cic_institute/subscriptions/'}},
  'callbackUrl': '',
  'consumerKey': 'f_9vaEvm3oxCnNe9Z4VoxRXZGwca',
  'consumerSecret': 'MTXYtMiwOmxNI2mRLrwOwG_9Vpwa',
  'description': '',
  'name': 'cic_institute',
  'tier': 'Unlimited'},
 'status': 'success',
 'version': '2.0.0-SNAPSHOT-rc3fad'}

In [21]:
key = rsp.json()['result']['consumerKey']

In [22]:
secret = rsp.json()['result']['consumerSecret']

In [23]:
# POST payload for generating a token using the password grant:
# scope will always be PRODUCTION.
data = {'username': username,
       'password': password,
       'grant_type': 'password',
       'scope': 'PRODUCTION'}
# note that authentication is technically HTTPBasicAuth with the OAuth client key and secret
rsp = requests.post('{}/token'.format(base_url), data=data, auth=(key, secret))
rsp.status_code

200

In [24]:
# check the response message:
rsp.json()

{'access_token': '6f8f0b94c413c8a7b5ba428bd50d4',
 'expires_in': 14400,
 'refresh_token': 'dafb6ab783b8edd2cfceda40a7f134',
 'scope': 'default',
 'token_type': 'bearer'}

In [26]:
# pull out the access and refresh tokens
access_token = rsp.json()['access_token']
refresh_token = rsp.json()['refresh_token']

In [27]:
# build the Authorization header in a headers dictionary
headers = {'Authorization': 'Bearer {}'.format(access_token)}

# make a request to the profiles service; the "me" endpoint is a special reserved word in Agave to indicate
# we want information about the associated token.
rsp = requests.get(url='{}/profiles/v2/me'.format(base_url), headers=headers)
rsp.status_code

200

In [28]:
# check the json response
rsp.json()

{'message': 'User details retrieved successfully.',
 'result': {'create_time': '20140515180254Z',
  'email': 'jstubbs@tacc.utexas.edu',
  'first_name': 'Joe',
  'full_name': 'jstubbs',
  'last_name': 'Stubbs',
  'mobile_phone': '',
  'phone': '',
  'status': '',
  'username': 'jstubbs'},
 'status': 'success',
 'version': '2.0.0-SNAPSHOT-rc3fad'}

Indeed, the profile belongs to me. We are now ready to interact with Agave's cloud storage.